In [181]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from requests import get
import re
import time

In [204]:
# get trail html
def get_soup(url):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'}
    page = get(url,headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

In [232]:
urlbase = 'https://www.trailforks.com/region/british-columbia/trails/'
trail_list_soup = get_soup(urlbase)
pagedata = trail_list_soup.find('ul', class_='paging-middle centertext').find_all('li')
lastpage = int(pagedata[-1].text)
url_list = []
for i in np.arange(lastpage):
    n = i+1
    if n == 1:
        url = urlbase
    else:
        url = urlbase + '?page=' + str(n)
    url_list.append(url)

In [233]:
trail_url = []
trail_name = []
region_url = []
region_name = []
for url in url_list:
    trail_list_soup = get_soup(url)
    all_table_soup = trail_list_soup.find_all('table')
    first_table = all_table_soup[0]
    trail_name_soup = first_table.tbody.find_all('td',class_='highlight')
    region_name_soup = first_table.tbody.find_all('td',class_='nowrap')

    for tr in trail_name_soup:
        trail_url.append(tr.find('a',href=True)['href'])
        trail_name.append(tr.find('a',href=True).text)

    for tr in region_name_soup:
        region_url.append(tr.find('a',href=True)['href'])
        region_name.append(tr.find('a',href=True).text)
        time.sleep(np.random.uniform(.5,1.5))



In [248]:
trail_id = []
for trail in trail_url:
    trail_id.append(trail.replace('https://www.trailforks.com/trails/','').replace('/',''))

df = pd.DataFrame(
    {
        'trail_id' : trail_id,
        'trail_url' : trail_url,
        'trail_name' : trail_name,
        'region_url' : region_url,
        'region_name' : region_name
        }
)
df.set_index('trail_id')
df.to_csv('/Users/briangraham/insight/trailrec/data/BC_trail_names')